# Tensor related torch examples
Tensor 관련된 기본적인 메소드들 모아놓음

In [2]:
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

## indexing

In [61]:
x = torch.arange(0, 10).reshape(2, 5)

In [94]:
x.shape

torch.Size([2, 5])

In [63]:
x[:2] # 0번째 차원에서 0~1번째 텐서 보자(=> 전체 텐서)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [65]:
x[1, 3] # 0번째 차원에서 2번째(2행), 1번째 차원에서 4번째(4열) 보자

tensor(8)

In [69]:
x[[0, 1], [3, 4]] # 1열 4행과 2행 5열 뽑자

tensor([3, 9])

In [71]:
idx = [[0, 1], [3, 4]] # 텐서를 인덱싱하는 건 일단 리스트라고 볼 수 있음. 그럼 list를 idx로 할당하고 인덱싱해도 똑같은 결과!
x[idx]

tensor([3, 9])

In [105]:
idx = torch.tensor([[0, 1], [3, 4]]) # 근데 같은 내용이라도 이게 텐서로 들어가있으면?
print("idx shape", idx.shape) # 그냥 행차원에서 인덱싱하는게 되고, 이때 만들어지는 텐서는 인덱싱한 shape을 따라간다.
x[idx] # x의 shape은 2행 5열인데 3, 4 행을 인덱싱하려고 해서 인덱스 에러

idx shape torch.Size([2, 2])


IndexError: index 3 is out of bounds for dimension 0 with size 2

In [106]:
x[0]

tensor([0, 1, 2, 3, 4])

In [107]:
x[torch.tensor(0)]

tensor([0, 1, 2, 3, 4])

In [108]:
x[0, 1]

tensor(1)

In [109]:
x[torch.tensor([0, 1])]

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [111]:
_ = x[torch.tensor([[[0, 1]]])] 
print(_) 
print(_.shape)

tensor([[[[0, 1, 2, 3, 4],
          [5, 6, 7, 8, 9]]]])
torch.Size([1, 1, 2, 5])


In [115]:
idx = torch.tensor([[0, 1], [1, 0]])
_ = x[idx]
print(idx.shape)
print(_)
print(_.shape)

torch.Size([2, 2])
tensor([[[0, 1, 2, 3, 4],
         [5, 6, 7, 8, 9]],

        [[5, 6, 7, 8, 9],
         [0, 1, 2, 3, 4]]])
torch.Size([2, 2, 5])


In [116]:
x[:, idx[1]]

tensor([[1, 0],
        [6, 5]])

그러면 원래 목표대로 list처럼 인덱싱을 하려면 어떻게 할까?<br>
아래와 같이 ,를 사용하게 명시적으로 주면 된다.

In [119]:
idx = torch.tensor([[0, 1], [3, 4]]) 
x[idx[0, :], idx[1, :]]

tensor([3, 9])

## .nonzero() 
텐서에서 요소가 0이 아닌 index 나옴

In [54]:
torch.tensor([[0, 1, 0], [1, 1, 1]]).nonzero()

tensor([[0, 1],
        [1, 0],
        [1, 1],
        [1, 2]])

## scatter
src, idx, tgt 텐서 세개가 있는데 src에 있는 idx만 가져와서 tgt에 적용하고 싶을 때 쓰는 메소드. 원핫으로 만들 때 많이 쓰는듯? 근데 one-hot은 메소드 따로 있긴 함..
https://yuyangyy.medium.com/understand-torch-scatter-b0fd6275331c<br>
Writes all values from the tensor src into self at the indices specified in the index tensor. For each value in src, its output index is specified by its index in src for dimension != dim and by the corresponding value in index for dimension = dim.

In [32]:
src = torch.arange(1, 11).reshape((2, 5))
src

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [33]:
index = torch.tensor([[0, 1, 2, 0]])
index

tensor([[0, 1, 2, 0]])

In [34]:
torch.zeros(3, 5, dtype=src.dtype).shape, src.shape, index.shape

(torch.Size([3, 5]), torch.Size([2, 5]), torch.Size([1, 4]))

In [35]:
torch.zeros(3, 5, dtype=src.dtype).scatter_(0, index, src)

tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])

In [36]:
# [1, 2, 3, 0, 0] -> [4, 2, 3, 0, 0]으로 된듯
torch.zeros(3, 5, dtype=src.dtype).scatter_(1, index, src)

tensor([[4, 2, 3, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])

In [37]:
src = torch.from_numpy(np.arange(1, 11)).float().view(2, 5)
print(src)

tensor([[ 1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10.]])


In [38]:
input_tensor = torch.zeros(3, 5)
print(input_tensor)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [39]:
index_tensor = torch.tensor([[0, 1, 2, 0, 0], [2, 0, 0, 1, 2]])
print(index_tensor)

tensor([[0, 1, 2, 0, 0],
        [2, 0, 0, 1, 2]])


In [40]:
## try to manually work out the result 
dim = 0
print(input_tensor.scatter_(dim, index_tensor, src))

tensor([[ 1.,  7.,  8.,  4.,  5.],
        [ 0.,  2.,  0.,  9.,  0.],
        [ 6.,  0.,  3.,  0., 10.]])


one-hot encoding할 때 쓰인다고 함
https://aigong.tistory.com/35

In [41]:
label = torch.Tensor([3, 6, 7]).long()
one_hot = torch.zeros(5, 10)

In [42]:
label = label.unsqueeze(1)

In [43]:
label.shape

torch.Size([3, 1])

In [44]:
one_hot.scatter_(1, label, 1)

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

# einsum

In [11]:
# trace
x = torch.arange(0, 16).reshape(4, 4)
print(x)
torch.einsum("ii", x)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])


tensor(30)

## where
mask_fill의 더 심화된 버전. src, tgt이 있을 때 src을 어떤 조건을 걸고 (boolean) True인 경우를 tgt element로 바꿔치기!

In [45]:
>>> x = torch.randn(3, 2)
>>> y = torch.ones(3, 2)

In [46]:
>>> x

tensor([[-1.6403,  0.8030],
        [ 2.4286, -0.2958],
        [ 1.3324,  0.3157]])

In [47]:
>>> torch.where(x > 0, x, y)

tensor([[1.0000, 0.8030],
        [2.4286, 1.0000],
        [1.3324, 0.3157]])

## repeat

In [ ]:
>>> x = torch.tensor([1, 2, 3])
>>> x.repeat(4, 2)

tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3]])

In [ ]:
torch.arange(0, 10).unsqueeze(0).repeat(3, 1)

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

## permute
(*dims) → Tensor
Returns a view of the original tensor with its dimensions permuted.

Parameters
*dims (int...) – The desired ordering of dimensions

In [ ]:

>>> x = torch.randn(2, 3, 5)
>>> x.size()
torch.Size([2, 3, 5])
>>> x.permute(2, 0, 1).size()
torch.Size([5, 2, 3])

torch.Size([5, 2, 3])

## product
- `*` : elementwise product 
- `@` : inner product or matmul 


In [ ]:
v = torch.Tensor([1, 2, 3])
print(v.shape)

torch.Size([3])


In [ ]:
v * v # hadamard product

tensor([1., 4., 9.])

In [ ]:
v * v.T # hadamard product

tensor([1., 4., 9.])

In [ ]:
v.T * v

tensor([1., 4., 9.])

In [ ]:
v @ v # inner product

tensor(14.)

In [ ]:
v.T @ v

tensor(14.)

In [ ]:
v @ v.T

tensor(14.)

In [ ]:
v.unsqueeze_(0)
v.shape

torch.Size([1, 3])

In [ ]:
v + v.T # broad casting 

tensor([[2., 3., 4.],
        [3., 4., 5.],
        [4., 5., 6.]])

In [ ]:
v * v.T # (1 x 3) * (3 x 1) -> broad casting 

tensor([[1., 2., 3.],
        [2., 4., 6.],
        [3., 6., 9.]])

In [ ]:
v.T * v # (3 x 1) * (1 x 3) -> broad casting

tensor([[1., 2., 3.],
        [2., 4., 6.],
        [3., 6., 9.]])

In [ ]:
v @ v.T # 1 x 3 @ 3 x 1 -> 1 x 1 

tensor([[14.]])

In [ ]:
v @ v # 1 x 3 @ 1 x 3 -> cannot multiply!

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3 and 1x3)

## bmm
Performs a batch matrix-matrix product of matrices stored in input and mat2.

input and mat2 must be 3-D tensors each containing the same number of matrices.

In [ ]:
input = torch.randn(10, 3, 4)
mat2 = torch.randn(10, 4, 5)
res = torch.bmm(input, mat2)
res.size()
torch.Size([10, 3, 5])

torch.Size([10, 3, 5])

## matmul
https://pytorch.org/docs/stable/generated/torch.matmul.html#torch.matmul

broadcasting된다는 것이 bmm과의 차이

In [ ]:
input = torch.randn(10, 3, 4)
mat2 = torch.randn(10, 4, 5)
res = input.matmul(mat2)
res.size()

torch.Size([10, 3, 5])

In [ ]:
>>> # batched matrix x broadcasted matrix
>>> tensor1 = torch.randn(10, 3, 4)
>>> tensor2 = torch.randn(4, 5)
>>> torch.matmul(tensor1, tensor2).size()

torch.Size([10, 3, 5])

In [ ]:
torch.bmm(tensor1, tensor2)

RuntimeError: batch2 must be a 3D tensor

## tril
하부 삼각형만 남기고 상부 삼각형은 0으로.. 뭐에 쓰냐 이거

In [ ]:
torch.tril(torch.Tensor(3, 4))

tensor([[ 4.4842e-44,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.1006e+23,  3.0787e-41,  0.0000e+00,  0.0000e+00],
        [ 7.0065e-45,  0.0000e+00,  2.3354e-07,  0.0000e+00]])

## masked_fill

In [ ]:
x = torch.arange(10)
mask = x < 5 

In [ ]:
mask

tensor([ True,  True,  True,  True,  True, False, False, False, False, False])

In [ ]:
x.masked_fill(mask, 3)

tensor([3, 3, 3, 3, 3, 5, 6, 7, 8, 9])

## view vs. transpose
웬만하면 transpose를 쓰는게 맞음. 다 펴주고 싶을 때 view(-1) 처럼 쓰는거 말고는 쓸 때 유의하자.

In [10]:
x = torch.arange(100)
x.view(2, 5, 10)

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]],

        [[50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
         [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
         [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
         [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]])

In [11]:
x = torch.arange(100)
x.view(5, 2, 10)

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]],

        [[40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]],

        [[60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74, 75, 76, 77, 78, 79]],

        [[80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
         [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]])

In [12]:
x = torch.arange(10)
xx = x.view(5, 2)

In [13]:
xx

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [14]:
xx.view(2, 5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [15]:
xx.transpose(1, 0)

tensor([[0, 2, 4, 6, 8],
        [1, 3, 5, 7, 9]])

## view vs. reshape
view는 contiguous한 텐서에 대해서만 가능하고 reshape은 contiguous한 tensor가 아니어도 연산 가능
-> view가 reshape보다 더 안전한 메소드인듯 하다

https://stackoverflow.com/questions/26998223/what-is-the-difference-between-contiguous-and-non-contiguous-arrays/26999092#26999092

In [16]:
x.reshape(2, 5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [17]:
x.view(2, 5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [18]:
x = x.view(2, 5).T
x.is_contiguous()

False

In [19]:
x.view(2, 5) # error가 난다

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [20]:
x.contiguous().view(2, 5)

tensor([[0, 5, 1, 6, 2],
        [7, 3, 8, 4, 9]])

In [21]:
x.reshape(2, 5)

tensor([[0, 5, 1, 6, 2],
        [7, 3, 8, 4, 9]])

## view vs flatten
view는 원래 순서대로가 아니면 .view(-1) 못하고 flatten은 상관없이 할 수 있음.
flatten이 더 편할 때는 4개 이상 차원이 있는데 앞 두개를 flatten하고 싶을 때 편할듯?

In [37]:
x = torch.arange(0, 10).reshape(2, 5)
print(x.view(-1))
x.T.view(-1)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [38]:
print(x.flatten(0, 1))
x.T.flatten(0, 1)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


tensor([0, 5, 1, 6, 2, 7, 3, 8, 4, 9])